In [ ]:
# Step 1: Install necessary libraries
!pip install pandas numpy scikit-learn nltk

# Step 2: Import libraries
import pandas as pd
import numpy as np
import nltk
import string
import zipfile
import requests
from io import BytesIO
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Step 3: Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as z:
    with z.open('SMSSpamCollection') as f:
        df = pd.read_csv(f, sep='\t', names=['label', 'message'])

# Step 4: Preprocess the data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text
    words = nltk.word_tokenize(text)

    # Remove stop words
    words = [word for word in words if word not in stopwords.words('english')]

    # Stemming
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]

    return ' '.join(words)

df['message'] = df['message'].apply(preprocess_text)

# Step 5: Feature extraction
tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(df['message']).toarray()
y = df['label'].map({'ham': 0, 'spam': 1}).values

# Step 6: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Train the classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Step 9: Make predictions
def predict_sms(text):
    text = preprocess_text(text)
    text_vec = tfidf.transform([text]).toarray()
    prediction = model.predict(text_vec)
    return 'spam' if prediction[0] == 1 else 'ham'

# Example
print(predict_sms("Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/123456 to claim now."))
print(predict_sms("Hey, are we still meeting for lunch tomorrow?"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.9802690582959641
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.85      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115

spam
ham
